In [1]:
import os

def crest_xyz_to_gjf(xyz_file, output_folder, method='B3LYP', basis_set='6-31G(d)', charge=0, multiplicity=1):   ###CAN CHANGE METHOD/BASIS SET HERE
    """
    Convert a CREST XYZ file with multiple conformers to separate Gaussian input (GJF) files,
    and store them in a specified output folder.

    Parameters:
    xyz_file (str or file-like object): Path to the input CREST XYZ file or a file-like object.
    output_folder (str): Path to the folder where the output GJF files will be saved.
    method (str): The computational method to use in Gaussian (default: B3LYP).
    basis_set (str): The basis set to use in Gaussian (default: 6-31G(d)).
    charge (int): The total charge of the molecule (default: 0).
    multiplicity (int): The multiplicity of the molecule (default: 1).
    """
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the XYZ file
    if isinstance(xyz_file, str):
        xyz_file = open(xyz_file, 'r')

    lines = xyz_file.readlines()
    xyz_file.close()

    index = 0
    num_lines = len(lines)
    conformer_index = 0

    while index < num_lines:
        # Read number of atoms
        num_atoms = int(lines[index].strip())
        index += 1

        # Read energy or comments
        energy_or_comment = lines[index].strip()
        index += 1

        # Extract atomic coordinates for the conformer
        atoms = []
        for _ in range(num_atoms):
            parts = lines[index].split()
            element = parts[0]
            x, y, z = map(float, parts[1:4])
            atoms.append((element, x, y, z))
            index += 1

        # Write the GJF file for the current conformer
        gjf_filename = os.path.join(output_folder, f"conformer_{conformer_index + 1}.gjf")
        with open(gjf_filename, 'w') as gjf_file:
            gjf_file.write('%mem=16GB\n')
            gjf_file.write('%nprocshared=16\n')
            gjf_file.write(f'# opt freq {method}/{basis_set} empiricaldispersion=gd3bj integral=ultrafine \n\n')    ###GAUSSAIN INPUT LINE/PARAMETERS 
            gjf_file.write(f'{energy_or_comment}\n\n')
            gjf_file.write(f'{charge} {multiplicity}\n')
            for atom in atoms:
                element, x, y, z = atom
                gjf_file.write(f'{element:>2}{x:>28.8f}{y:>14.8f}{z:>14.8f}\n')
            gjf_file.write('\n')

        conformer_index += 1

# Example usage:
xyz_file_path = 'test folder/crest_conformer.xyz'   ###PUT PATH TO XYZ FILE HERE
output_folder = 'test1_gjf_files'                   ###PATH TO OUTPUT FOLDER 
crest_xyz_to_gjf(xyz_file_path, output_folder)